# Demo notbook TSP

Demo notebook comparing classical approaches to the travellign salesman problem 
(TSP) with with the quantum Grover algorithm utilizing quantum phase estimation
and quantum adiabatic annealing (QAA).

In [1]:
import matplotlib.pyplot as plt
from src.simulated_annealer import SimulatedAnnealer

In [2]:
sim = SimulatedAnnealer()

In [3]:
_ = sim.set_cities_coordinates(nr_of_cities=30)

In [4]:
#_ = sim.solve_tsp(t_0=1, dt=0.0001)

In [5]:
#%matplotlib widget
#sim.plot_last_optimization_history()

In [6]:
from src.grover import Grover
from src.brute_force_solver import BruteForceSolver
from src import helpers

In [7]:
grover = Grover()
brute = BruteForceSolver()
cities = grover.set_cities_coordinates(4, 20)
brute.cities = cities

In [8]:
solution_brute = brute.solve_tsp(cities)

In [9]:
grover.calculate_distances(cities)

{'10': 10, '20': 15, '21': 19, '30': 6, '31': 14, '32': 20}

In [10]:
scaled_results = []
for size in range(8,32,2):
    paths = []
    print(size)
    for i in range(30):
        cities = grover.set_cities_coordinates(4, size)
        brute.cities = cities
        solution_brute = brute.solve_tsp(cities)
        solution_grover = grover.solve_tsp(cities)
        correct = helpers.sequences_are_equal(solution_brute, solution_grover)
        run_result = [correct, solution_brute, solution_grover]
        paths.append(run_result)
        #print(paths[-1])
    scaled_results.append([result[0] for result in paths])

8
10
12


In [ ]:
print(len(scaled_results))
for i in range(len(scaled_results)):
    print(sum(scaled_results[i]))
sc_res = { 
    8: 10,
    10: 19,
    12: 13,
    14: 15,
    16: 21,
    18: 13,
    20: 10,
    22: 20,
    24: 13,
    26: 22,
    28: 22,
}

NameError: name 'scaled_results' is not defined

In [ ]:
grover.qc.draw()

┌───┐                                                            »
 q1479_0: ┤ H ├──■────────────────────────■─────────■──────────────────────»
          ├───┤  │                        │         │                      »
 q1479_1: ┤ H ├──┼────■───────────────────┼────■────┼────■─────────────────»
          ├───┤┌─┴─┐  │  ┌───┐     ┌───┐┌─┴─┐  │    │    │                 »
 q1479_2: ┤ H ├┤ X ├──┼──┤ X ├──■──┤ X ├┤ X ├──┼────┼────┼─────────────────»
          ├───┤└───┘┌─┴─┐├───┤  │  ├───┤└───┘┌─┴─┐  │    │                 »
 q1479_3: ┤ H ├─────┤ X ├┤ X ├──■──┤ X ├─────┤ X ├──┼────┼─────────────────»
          ├───┤     └───┘└───┘  │  └───┘     └───┘┌─┴─┐  │  ┌───┐     ┌───┐»
 q1479_4: ┤ H ├─────────────────┼─────────────────┤ X ├──┼──┤ X ├──■──┤ X ├»
          ├───┤                 │                 └───┘┌─┴─┐├───┤  │  ├───┤»
 q1479_5: ┤ H ├─────────────────┼──────────────────────┤ X ├┤ X ├──■──┤ X ├»
          └───┘               ┌─┴─┐┌───┐               └───┘└───┘  │  └───┘»
 q1479_6: ────────────────────┤ X ├┤ X ├───────────────────────────┼───────»
                              └───┘└───┘                         ┌─┴─┐┌───┐»
 q1479_7: ───────────────────────────────────────────────────────┤ X ├┤ X ├»
                                                                 └───┘└───┘»
 q1479_8: ─────────────────────────────────────────────────────────────────»
                                                                           »
 q1479_9: ─────────────────────────────────────────────────────────────────»
                                                                           »
q1479_10: ─────────────────────────────────────────────────────────────────»
                                                                           »
q1479_11: ─────────────────────────────────────────────────────────────────»
                                                                           »
q1479_12: ─────────────────────────────────────────────────────────────────»
                                                                           »
q1479_13: ─────────────────────────────────────────────────────────────────»
                                                                           »
q1479_14: ─────────────────────────────────────────────────────────────────»
                                                                           »
q1479_15: ─────────────────────────────────────────────────────────────────»
                                                                           »
q1479_16: ─────────────────────────────────────────────────────────────────»
                                                                           »
q1479_17: ─────────────────────────────────────────────────────────────────»
                                                                           »
q1479_18: ─────────────────────────────────────────────────────────────────»
                                                                           »
q1479_19: ─────────────────────────────────────────────────────────────────»
                                                                           »
q1479_20: ─────────────────────────────────────────────────────────────────»
                                                                           »
q1479_21: ─────────────────────────────────────────────────────────────────»
                                                                           »
q1479_22: ─────────────────────────────────────────────────────────────────»
                                                                           »
q1479_23: ─────────────────────────────────────────────────────────────────»
          ┌───┐                                                            »
q1479_24: ┤ X ├────────────────────────────────────────────────────────────»
          └───┘                                                            »
   c29: 6/═════════════════════════════════════════════════════════════════»
      